In [689]:
# Construct the bigram training set

import torch
import torch.nn.functional as F

words = open("../data/names.txt", "r").read().splitlines()
alphabet = sorted(list(set("".join(words))))
LIMITER="."

atoi = {s:i+1 for i,s in enumerate(alphabet)}
atoi[LIMITER] = 0

itoa = {i:s for s,i in atoi.items()}

C = len(atoi)
N = torch.zeros((C, C), dtype=torch.int32)

xs, ys = [], []

for w in words:
    chs = [LIMITER] + list(w) + [LIMITER]
    for ch1, ch2, in zip(chs, chs[1:]):
        i1 = atoi[ch1]
        i2 = atoi[ch2]
        xs.append(i1)
        ys.append(i2)

xs = torch.tensor(xs, dtype=torch.long)
ys = torch.tensor(ys, dtype=torch.long)

# Total Bigrams
total = xs.nelement()

In [690]:
# Intialize a 27x27 weight matrix. 27 neurons and 27 activations for each letter of the alphabet
W = torch.randn(C, C, requires_grad=True).float()

# Vectorize Onehot
xenc = F.one_hot(xs, num_classes=C).float()

In [691]:
for i in range(500):
    # === Forward ===
    logits = xenc @ W

    c = logits.exp()
    probs = c / c.sum(1, keepdim=True)

    regularization_term = (0.01 * (W**2).mean())
    loss = -(probs[torch.arange(xs.nelement()), ys].log().mean())

    total_loss = loss + regularization_term
    print(total_loss)

    # === Backward ===
    W.grad = None
    total_loss.backward()

    W.data -= 10 * W.grad

tensor(3.8563, grad_fn=<AddBackward0>)
tensor(3.7637, grad_fn=<AddBackward0>)
tensor(3.6800, grad_fn=<AddBackward0>)
tensor(3.6035, grad_fn=<AddBackward0>)
tensor(3.5333, grad_fn=<AddBackward0>)
tensor(3.4686, grad_fn=<AddBackward0>)
tensor(3.4088, grad_fn=<AddBackward0>)
tensor(3.3536, grad_fn=<AddBackward0>)
tensor(3.3027, grad_fn=<AddBackward0>)
tensor(3.2558, grad_fn=<AddBackward0>)
tensor(3.2129, grad_fn=<AddBackward0>)
tensor(3.1735, grad_fn=<AddBackward0>)
tensor(3.1376, grad_fn=<AddBackward0>)
tensor(3.1048, grad_fn=<AddBackward0>)
tensor(3.0748, grad_fn=<AddBackward0>)
tensor(3.0473, grad_fn=<AddBackward0>)
tensor(3.0221, grad_fn=<AddBackward0>)
tensor(2.9989, grad_fn=<AddBackward0>)
tensor(2.9775, grad_fn=<AddBackward0>)
tensor(2.9576, grad_fn=<AddBackward0>)
tensor(2.9391, grad_fn=<AddBackward0>)
tensor(2.9218, grad_fn=<AddBackward0>)
tensor(2.9057, grad_fn=<AddBackward0>)
tensor(2.8905, grad_fn=<AddBackward0>)
tensor(2.8763, grad_fn=<AddBackward0>)
tensor(2.8629, grad_fn=<A

In [692]:

g = torch.Generator().manual_seed(1234)
for i in range(10):
    out = []
    ix = 0
    while(True):
        xenc = F.one_hot(torch.tensor([ix]), num_classes=27).float()
        logits = xenc @ W
        lc = logits.exp()
        probs = lc / lc.sum(1, keepdim=True)

        ix = torch.multinomial(probs, num_samples=1, replacement=True, generator=g).item()

        ch = itoa[ix]
        out.append(ch)
        if ch == LIMITER:
            break

    print(("".join(out))[:-1])

kyason
kanana
milaveela
ra
drisan
lurtolisyrsh
tobay
n
myaeahare
chucinailelulelone
